# Build AML Pipeline with azureml modules

In this tutorial you will learn how to work with Azure ML module:

1. Setup enrivonment - install module CLI and module/pipeline SDK
2. Register a few sample modules into your aml workspace using CLI
3. Use module/pipeline SDK to create a pipeline with modules registered in step 2

## Prerequisite
* Install Azure CLI, please follow [the Azure CLI installation instructions](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest) to install.
* Install docker desktop from [here](https://www.docker.com/products/docker-desktop) 

## Setup environment
* Install Azure CLI AML extension which includes the _module_ command group
* Install Azure ML SDK including the APIs to work with _module_ and _pipeline_

In [ ]:
CLI_SDK_VERSION=31132438

In [ ]:
!az extension remove -n azure-cli-ml 

# Install local version of azure-cli-ml (which includes `az ml module` commands)
!az extension add --source https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/$CLI_SDK_VERSION/azure_cli_ml-0.1.0.$CLI_SDK_VERSION-py3-none-any.whl --pip-extra-index-urls https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/$CLI_SDK_VERSION --yes 

In [ ]:
# Verify the availability of `az ml module` commands
#!az ml pipeline -h
!az ml module -h

In [ ]:
# Install azureml-sdk with Pipeline, Module
# Important! After install succeed, need to restart kernel

#%config IPCompleter.greedy=True 
#!pip install azureml-pipeline-wrapper[notebooks]==0.1.0.$CLI_SDK_VERSION --extra-index-url https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/$CLI_SDK_VERSION --user --upgrade 
#!pip install azureml-core==0.1.0.$CLI_SDK_VERSION --extra-index-url https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/$CLI_SDK_VERSION --user --upgrade 

!pip install azure-ml-component[notebooks] --upgrade

## Register azureml module

You can manage AML module through [azure-cli-ml](https://aka.ms/moduledoc) or [ml.azure.com](https://ml.azure.com/). <br>

Module could be registered from:
- local path
- public Github url
- Azure DevOps build artifacts

Azureml module support multiple module type:
- Basic python module
- Mpi module
- Parallel run module
- Hdi module (pending on backend support)

In [ ]:
# you need to configure your ws information here

subscription_id = '74eccef0-4b8d-4f83-b5f9-fa100d155b22' #'4aaa645c-5ae2-4ae9-a17a-84b9023bc56a'#'74eccef0-4b8d-4f83-b5f9-fa100d155b22'
workspace_name = 'lisal-amlservice' #'itp-pilot'#'kubeflow_ws_2' #'lisal-amlservice'
resource_group = 'lisal-dev' #'itp-pilot-ResGrp'#'kubeflow-demo' #'lisal-dev'

# Specify available aml compute in workspace
pipeline_compute = 'always-on-ds2v2' #'k80-16-a'#'kubeflow-aks' #'always-on-ds2v2'


In [ ]:
# Configure your aml workspace 

!az login 
!az account set -s $subscription_id 
!az ml folder attach -w $workspace_name -g $resource_group 

# Configure global .amlignore, it's designed for register module from local development environment
# !az configure --defaults module_amlignore_file=./.amlignore

In [ ]:
# Register azureml modules from github url

!az ml component create --file https://github.com/lisagreenview/hello-aml-modules/blob/master/train-score-eval/train.yaml --label default
!az ml component create --file https://github.com/lisagreenview/hello-aml-modules/blob/master/train-score-eval/score.yaml --label default
!az ml component create --file https://github.com/lisagreenview/hello-aml-modules/blob/master/train-score-eval/eval.yaml --label default
!az ml component create --file https://github.com/lisagreenview/hello-aml-modules/blob/master/train-score-eval/compare2.yaml --label default

In [ ]:
# list available custom module in aml workspace
!az ml component list -o table 

## Create pipeline
You can build pipeline through SDK experience, or drag-n-drop way through [Designer](https://ml.azure.com/visualinterface?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/kubeflow-demo/workspaces/kubeflow_ws_1&flight=cm,nml,newGraphDetail,newGraphAuthoring,all&tid=72f988bf-86f1-41af-91ab-2d7cd011db47) in workspace portal

The new SDK:
* Symplified the syntax to provide consistent experience with drag-n-drop
* Support intellisense and docstring, free you to work with dict all the time
* Support creating a pipeline with unpublished module

In [ ]:
from azureml.core import Workspace, Dataset
from azure.ml.component import Component

ws = Workspace.get(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group)

# get modules
# load module from local unpublished module
train_module_func = Component.load(ws, name='microsoft.com.azureml.samples.train')
score_module_func = Component.load(ws, name='microsoft.com.azureml.samples.score')
eval_module_func = Component.load(ws, name='microsoft.com.azureml.samples.evaluate')
compare_module_func = Component.load(ws, name='microsoft.com.azureml.samples.compare_2_models')

"""
# load module from ws published modules
train_module_func = Component.from_yaml(ws, yaml_file='./train-score-eval/train.yaml')
score_module_func = Component.from_yaml(ws, yaml_file='./train-score-eval/score.yaml')
eval_module_func = Component.from_yaml(ws, yaml_file='./train-score-eval/eval.yaml')
compare_module_func = Component.from_yaml(ws, yaml_file='./train-score-eval/compare2.yaml')
"""

# get dataset
training_data_name = 'training_data'
test_data_name = 'test_data'

if training_data_name not in ws.datasets:
    print('Registering a training dataset for sample pipeline ...')
    train_data = Dataset.File.from_files(path=['https://dprepdata.blob.core.windows.net/demo/Titanic.csv'])
    train_data.register(workspace = ws, 
                              name = training_data_name, 
                              description = 'Training data (just for illustrative purpose)')
    print('Registerd')
else:
    train_data = ws.datasets[training_data_name]
    print('Training dataset found in workspace')

if test_data_name not in ws.datasets:
    print('Registering a test dataset for sample pipeline ...')
    test_data = Dataset.File.from_files(path=['https://dprepdata.blob.core.windows.net/demo/Titanic.csv'])
    test_data.register(workspace = ws, 
                          name = test_data_name, 
                          description = 'Test data (just for illustrative purpose)')
    print('Registered')
else:
    test_data = ws.datasets[test_data_name]    
    print('Test dataset found in workspace')


### dsl pipeline 
* 'Pipeline parameter' is exposed as pipeline function input parameter
* Pipeline output is the return of pipeline function

### module function
* module input can be set through set_inputs() or module initialization function
* module parameter can be set through set_parameter() or module initialization function
* module runsetting including compute, datastore, data mode and other runtime parameter are set through runsettings.configure()


In [ ]:
from azure.ml.component import dsl

# define a sub pipeline
@dsl.pipeline(name = 'train-score-eval', 
              description = 'train model and evaluate model perf')
def training_pipeline(input_data, test_data, learning_rate):
    train = train_module_func()

    train.set_inputs(training_data=input_data, learning_rate=learning_rate, max_epochs=5)

    score = score_module_func(
        model_input=train.outputs.model_output, 
        test_data=test_data)

    eval = eval_module_func(scoring_result=score.outputs.score_output)
    
    return {'eval_output': eval.outputs.eval_output, 'model_output': train.outputs.model_output}

In [ ]:
# define pipeline with sub pipeline
@dsl.pipeline(name = 'A dummy pipeline', 
              description = 'select best model trained with different learning rate',
              default_compute_target = pipeline_compute)
def dummy_automl_pipeline(input_data, test_data, learning_rate_1, learning_rate_2):
    train_and_evalute_model1 = training_pipeline(input_data, test_data, learning_rate_1)
    train_and_evalute_model2 = training_pipeline(input_data, test_data, learning_rate_2)
    
# create pipeline
my_pipeline = dummy_automl_pipeline(input_data=train_data, test_data=test_data, learning_rate_1=0.01, learning_rate_2=0.02)

In [ ]:
# validate pipeline and visualize the graph
my_pipeline.validate()

### Submit a pipeline run

In [ ]:
# save as a draft, then you can continue to modify the pipeline in AML Studio Designer page
my_pipeline._save(experiment_name = 'pipeline-with-azureml-module')

In [ ]:
# pipeline parameter can be override when submit pipeline
from azure.ml.component._version import VERSION

run = my_pipeline.submit(experiment_name='pipeline-with-azureml-component', tags={'mode':'SDK/dsl','SDK-version':f'{VERSION}'}, pipeline_parameters={'input_data':train_data,'test_data':test_data, 'learning_rate_1': 0.05, 'learning_rate_2':0.06})

In [ ]:
# define pipeline with sub pipeline
@dsl.pipeline(name = 'A dummy pipeline', 
              description = 'select best model trained with different learning rate',
              default_compute_target = pipeline_compute)
def dummy_automl_pipeline(input_data, test_data, learning_rate_1, learning_rate_2):
    train_and_evalute_model1 = training_pipeline(input_data, test_data, learning_rate_1)
    train_and_evalute_model2 = training_pipeline(input_data, test_data, learning_rate_2)
    
    compare = compare_module_func(
        model1=train_and_evalute_model1.outputs.model_output, 
        eval_result1=train_and_evalute_model1.outputs.eval_output,
        model2=train_and_evalute_model2.outputs.model_output,
        eval_result2=train_and_evalute_model2.outputs.eval_output
    )

    return {'best_model': compare.outputs.best_model, 'best_result': compare.outputs.best_result}

# create pipeline
my_pipeline_2 = dummy_automl_pipeline(input_data=train_data, test_data=test_data, learning_rate_1=0.25, learning_rate_2=0.15)
run = my_pipeline_2.submit(experiment_name='pipeline-with-azureml-component', tags={'mode':'SDK/dsl','SDK-version':f'{VERSION}'})
run.wait_for_completion()

In [ ]:
my_pipeline.diff(my_pipeline_2)

### Publish and consume pipeline endpoint

In [ ]:
from azure.ml.component._endpoint import PipelineEndpoint

# publish pipeline endpoint
pipeline_endpoint = PipelineEndpoint.publish(workspace=ws, name="Train-Score-Eval-Compare",
                                             pipeline=my_pipeline_2, description="Train-Score-Eval-Compare", 
                                             set_as_default=True)

In [ ]:
# get pipeline endpoint by name
my_pipeline_endpoint = PipelineEndpoint.get(workspace=ws, name="Train-Score-Eval-Compare")

# trigger published pipeline, assign value to pipeline parameter
run_pipeline_endpoint = my_pipeline_endpoint.submit(experiment_name="PipelineEndpoint-Train-Score-Eval-Compare", pipeline_parameters={'input_data':test_data,'test_data':train_data, 'learning_rate_1':0.03, 'learning_rate_2':0.04})

### Export pipeline yaml

In [ ]:
pipeline.export(directory=os.path.join('.', 'export'))

### Load an unpublished module, and test if locally
* Support load module from local or github 
* Support use the module without publish it to aml ws
* Use module.run() to test the module locally

In [ ]:
from azure.ml.component import Component

# load unregistered module from github
compare_2_func = Component.from_yaml(ws, yaml_file='https://github.com/lisagreenview/hello-aml-modules/blob/master/train-score-eval/compare2.yaml')
help(compare_2_func)

In [ ]:
# create a module
# you need to prepare local test data to initialize the module
compare = compare_2_func(model1 = 'D:\\work\\code\\train-score-eval\\compare\\data\\compare_entry\\inputs\\model1',
                         eval_result1 = 'D:\\work\\code\\train-score-eval\\compare\\data\\compare_entry\\inputs\\eval_result1',
                         model2 = 'D:\\work\\code\\train-score-eval\\compare\\data\\compare_entry\\inputs\\model2',
                         eval_result2 = 'D:\\work\\code\\train-score-eval\\compare\data\\compare_entry\\inputs\\eval_result2')

compare.run(use_docker=True, track_run_history=True)

### Create a new pipeline with unregistered module

new pipeline = dummy_automl_pipeline + copy_file_func

In [ ]:
@dsl.pipeline(name='pipeline-with-azureml-module',default_compute_target = pipeline_compute)
def add_copy_file():
    compare_pipeline = dummy_automl_pipeline(input_data=train_data, test_data=test_data)
    copy_file_node = copy_file_func(input_folder=compare_pipeline.outputs.best_result)
    copy_file_node.runsettings.configure(node_count=1)

    return {**copy_file_node.outputs}

new_pipeline = add_copy_file()

### Run pipeline locally

* pipeline.run() support BasicPythonModule, ParallelRunModule and MpiModule 
* Local run and module node log will be uploaded and recorded in aml ws, and the running status will also be synced back to aml ws
* For mpi module, local run only support image with openmpi, intelmpi not support

In [ ]:
run = new_pipeline.run(experiment_name = 'pipeline-local_run', show_output = True, show_graph = True)
run

In [ ]:
# submit to aml ws
run = new_pipeline.submit(experiment_name='pipeline-with-azureml-module', tags={'mode':'module-SDK','SDK-version':f'{CLI_SDK_VERSION}'}, description='add parallel copy file module')
run.wait_for_completion()